## Import library

In [84]:
%run bert
%run utils.ipynb
import re
import string
import numpy as np
import pandas as pd
import sklearn as sk 
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score,mean_absolute_error,mean_squared_error,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import optuna

In [78]:
loader = DataLoader()
d = loader.GetData('ASAP')
d = d.loc[d['essay_set'] ==1]
d.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,NaN,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4.0,3.0,NaN,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5.0,5.0,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    # Remove non-Roman characters
    text = re.sub("([^\x00-\x7F])+", " ", text)    
    text = re.sub("[\n\r\t]", " ", text)   
    return text
t = [clean_text(i) for i in d['essay']]

In [88]:
EmbeddingFetcher = GetBERTEmbeddings(d['essay'],'model/deberta-v3-large')
EmbeddingFetcher.inf(stop=1500,SeqLen = 512 )

tokenized


c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\deberta_v2\modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\deberta_v2\modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\deberta_v2\modeling_deberta_v2.py:852: UserWarning: To copy construct from a tensor, it is recommended 

0/1500, run:0
10/1500, run:0
20/1500, run:0
30/1500, run:0
40/1500, run:0
50/1500, run:0
60/1500, run:0
70/1500, run:0
80/1500, run:0
90/1500, run:0
100/1500, run:0
110/1500, run:0
120/1500, run:0
130/1500, run:0
140/1500, run:0
150/1500, run:0
160/1500, run:0
170/1500, run:0
180/1500, run:0
190/1500, run:0
200/1500, run:0
210/1500, run:0
220/1500, run:0
230/1500, run:0
240/1500, run:0
250/1500, run:0
260/1500, run:0
270/1500, run:0
280/1500, run:0
290/1500, run:0
300/1500, run:0
310/1500, run:0
320/1500, run:0
330/1500, run:0
340/1500, run:0
350/1500, run:0
360/1500, run:0
370/1500, run:0
380/1500, run:0
390/1500, run:0
400/1500, run:0
410/1500, run:0
420/1500, run:0
430/1500, run:0
440/1500, run:0
450/1500, run:0
460/1500, run:0
470/1500, run:0
480/1500, run:0
490/1500, run:0
500/1500, run:0
510/1500, run:0
520/1500, run:0
530/1500, run:0
540/1500, run:0
550/1500, run:0
560/1500, run:0
570/1500, run:0
580/1500, run:0
590/1500, run:0
600/1500, run:0
610/1500, run:0
620/1500, run:0
630

In [89]:
del EmbeddingFetcher.model
torch.cuda.empty_cache()

In [97]:
x = EmbeddingFetcher.GetEmbeddings('CLS')

In [98]:
x = [np.array(i).reshape(1024) for i in x]
TrainX,TestX = np.array(x[:1200]),np.array(x[1200:])

In [99]:
le = LabelEncoder()
le.fit(d['domain1_score'])
y = le.transform(d['domain1_score']) 
TrainY,TestY = y[:1200],y[1200:]

In [100]:
model = xgb.XGBClassifier(tree_method="hist",objective='multi:softmax',num_class=11)
model.fit(TrainX,TrainY)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_class=11, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=0, ...)

In [101]:
PredY = model.predict(TestX)

In [102]:
accuracy_score(TestY,PredY)

0.4854202401372213

In [104]:
cohen_kappa_score(TestY,PredY,weights='quadratic')

0.7619823173569101

In [80]:
#x = EmbeddingFetcher.GetEmbeddings('CLS')
#x = [np.array(i).reshape(768) for i in x]
#TrainX,TestX = x[:2800],x[2800:]
def objective():
    params = {"random_state":42,          
        'learning_rate' : 0.05,   
        "n_estimators": 1000,                 
        "max_depth" : 8,
        "alpha" : 0.95,
        "tree_method": 'gpu_hist', 
        "predictor": "gpu_predictor",
        "objective":'multi:softmax',
        "num_class":11
    }

    model = xgb.XGBClassifier(**params)
    model.fit(TrainX,TrainY)
    PredY = model.predict(TestX)
    e = accuracy_score(TestY,PredY)
    k = cohen_kappa_score(TestY,PredY)
    return e,k
objective()

(0.48713550600343053, 0.2988250531964105)